# 03. Data Loading, Quality Control, and Normalization
### From Quantification to Seurat Object

---

## 1. Overview
We have successfully processed 14 samples and generated gene expression counts (`quant.sf` files) using Salmon. 
Now, we switch our analysis environment to **R** to:
1.  **Import** the quantification data.
2.  **Annotate** the samples with biological metadata (Species, Strain, Condition).
3.  **Create a Seurat Object** for downstream analysis.
4.  **Perform Quality Control (QC)** to check the sequencing depth.

## 2. The Dataset (Metadata)
We need to map our **SRR Accessions** to meaningful biological labels:

| Accession | Species | Condition | Replicate |
|:---|:---|:---|:---:|
| **SRR25445867** | *P. aeruginosa* | WT (Biofilm) | 1 |
| **SRR25445868** | *P. aeruginosa* | WT (Biofilm) | 2 |
| **SRR25445869** | *P. aeruginosa* | Mutant ($\Delta\Delta$) | 1 |
| **SRR25445870** | *P. aeruginosa* | Mutant ($\Delta\Delta$) | 2 |
| **SRR25445871** | *S. aureus* | WT | 1 |
| **SRR25445872** | *S. aureus* | WT | 2 |
| **SRR25445873** | *S. aureus* | Mutant | 1 |
| **SRR25445874** | *S. aureus* | Mutant | 2 |
| **SRR25445875** | *E. coli* | WT | 1 |
| **SRR25445876** | *E. coli* | WT | 2 |
| **SRR25445877** | *E. coli* | Mutant | 1 |
| **SRR25445878** | *E. coli* | Mutant | 2 |
| **SRR25445879** | *E. coli* | Mutant (Biofilm) | 1 |
| **SRR25445880** | *E. coli* | Mutant (Biofilm) | 2 |

In [ ]:
# 1. Install and Load Libraries
# We check if packages exist, if not we install them.

# Install BiocManager to get bioinformatics tools
if (!requireNamespace("BiocManager", quietly = TRUE)) install.packages("BiocManager")

# Install tximport (reads Salmon data)
if (!requireNamespace("tximport", quietly = TRUE)) BiocManager::install("tximport")

# Install Seurat (analysis toolkit)
if (!requireNamespace("Seurat", quietly = TRUE)) install.packages("Seurat")

# Load the libraries
library(tximport)
library(Seurat)
library(dplyr)

print("✅ Libraries loaded successfully!")

In [ ]:
# 2. Define Metadata & File Paths (Corrected Path)

samples <- c(
  "SRR25445867", "SRR25445868", # PAO1 WT
  "SRR25445869", "SRR25445870", # PAO1 Mutant
  "SRR25445871", "SRR25445872", # USA300 WT
  "SRR25445873", "SRR25445874", # USA300 Mutant
  "SRR25445875", "SRR25445876", # MG1655 WT
  "SRR25445877", "SRR25445878", # MG1655 Mutant
  "SRR25445879", "SRR25445880"  # MG1655 Mutant Biofilm
)

# Create Metadata
metadata <- data.frame(
  sample_id = samples,
  species = c(rep("P. aeruginosa", 4), rep("S. aureus", 4), rep("E. coli", 6)),
  condition = c(rep("WT", 2), rep("Mutant", 2), 
                rep("WT", 2), rep("Mutant", 2),
                rep("WT", 2), rep("Mutant", 2), rep("Mutant_Biofilm", 2)),
  replicate = rep(c("Rep1", "Rep2"), 7)
)
rownames(metadata) <- samples

# 🔥🔥 Correction is HERE: Added "../" at the beginning 🔥🔥
files <- file.path("../processed_data/salmon_quant", samples, "quant.sf")
names(files) <- samples

# Check again
if(all(file.exists(files))) {
  print("✅ Great! All 14 quantification files found.")
} else {
  print("❌ Error: Some files are missing!")
  print(files[!file.exists(files)])
}

In [ ]:
# 3. Import Data Separately & Merge (The Fix)

print("🔄 Processing Species Separately...")

# --- A. Pseudomonas aeruginosa (Samples 1-4) ---
print("1. Importing P. aeruginosa...")
files_pa <- files[1:4]
txi_pa <- tximport(files_pa, type = "salmon", txOut = TRUE)
mat_pa <- txi_pa$counts
# Prefix gene names to avoid confusion
rownames(mat_pa) <- paste0("PAO1_", rownames(mat_pa))
# Create Object
obj_pa <- CreateSeuratObject(counts = mat_pa, project = "PAO1")


# --- B. S. aureus (Samples 5-8) ---
print("2. Importing S. aureus...")
files_sa <- files[5:8]
txi_sa <- tximport(files_sa, type = "salmon", txOut = TRUE)
mat_sa <- txi_sa$counts
rownames(mat_sa) <- paste0("USA300_", rownames(mat_sa))
obj_sa <- CreateSeuratObject(counts = mat_sa, project = "USA300")


# --- C. E. coli (Samples 9-14) ---
print("3. Importing E. coli...")
files_ec <- files[9:14]
txi_ec <- tximport(files_ec, type = "salmon", txOut = TRUE)
mat_ec <- txi_ec$counts
rownames(mat_ec) <- paste0("MG1655_", rownames(mat_ec))
obj_ec <- CreateSeuratObject(counts = mat_ec, project = "MG1655")


# --- D. Merge into One Big Object ---
print("4. Merging all data...")
# Seurat's merge function handles different genes automatically (fills missing with 0)
bact_seurat <- merge(obj_pa, y = c(obj_sa, obj_ec))

# --- E. Add Metadata ---
# Add our biological info to the merged object
# Ensure the order matches (Seurat usually keeps order, but let's be safe)
bact_seurat <- AddMetaData(bact_seurat, metadata = metadata)

# Verify
print("✅ SUCCESS! Final Object Created:")
print(bact_seurat)

In [ ]:
# Check the stats for each species
print("--- Average Reads per Species ---")
tapply(bact_seurat$nCount_RNA, bact_seurat$species, summary)

print("--- Average Genes Detected per Species ---")
tapply(bact_seurat$nFeature_RNA, bact_seurat$species, summary)

In [ ]:
# 3. Import Data (Filtering for Valid Clean Samples)

print("🔄 Processing Valid Samples (E. coli Only)...")

# We only load E. coli samples because Pseudomonas/S. aureus require adapter trimming
valid_samples <- c("SRR25445875", "SRR25445876", # WT
                   "SRR25445877", "SRR25445878", # Mutant
                   "SRR25445879", "SRR25445880") # Biofilm

# Define paths for valid samples (Going back one folder to processed_data)
# Note the "../" because we are in the notebooks folder
valid_files <- file.path("../processed_data/salmon_quant", valid_samples, "quant.sf")
names(valid_files) <- valid_samples

# Verification
if(!all(file.exists(valid_files))) {
    print("❌ Error: Files not found. Check paths.")
    print(valid_files[!file.exists(valid_files)])
} else {
    print("✅ All E. coli files found. Importing...")
    
    # Import using tximport
    txi <- tximport(valid_files, type = "salmon", txOut = TRUE)
    counts <- txi$counts
    
    # Rename genes (Add MG1655 prefix)
    rownames(counts) <- paste0("MG1655_", rownames(counts))
    
    # Create Seurat Object
    # Filter metadata to match only these 6 samples
    meta_filtered <- metadata[valid_samples, ]
    
    bact_seurat <- CreateSeuratObject(counts = counts, 
                                      meta.data = meta_filtered, 
                                      project = "E_coli_Analysis")
    
    print("✅ SUCCESS! Seurat Object created for E. coli.")
    print(bact_seurat)
}

In [ ]:
# 4. Quality Control & Normalization

print("📊 Generating QC Plots for E. coli...")

# Visualize Sequencing Depth & Gene Counts
# - nCount_RNA: Total reads (عمق القراءة)
# - nFeature_RNA: Detected genes (عدد الجينات المكتشفة)

vln <- VlnPlot(bact_seurat, 
               features = c("nFeature_RNA", "nCount_RNA"), 
               ncol = 2, 
               pt.size = 2, 
               group.by = "condition") # Group by Condition (WT vs Mutant)

print(vln)

# Normalize the data
# This is crucial so we can compare Sample A vs Sample B fairly
print("⚖️ Normalizing Data...")
bact_seurat <- NormalizeData(bact_seurat, normalization.method = "LogNormalize", scale.factor = 10000)

print("✅ Data Normalized. Notebook 03 Complete!")